# Image and text Extraction

In [1]:
# classes: 0-Caption 1-Handwritting 2-Image 3-Text

from image_processor import extract_image
from text_processor import extract_text
from clip import CLIPInstance
from ultralytics import YOLO
from data_format import DataFormat

import json, os

data_path = "../dataset/facticia_3_2/"
export_path = "../dataset/results/"

filename = "4c76a3e4-ohcbh_cf_erl8_204"

image_path = f"{data_path}/images/{filename}.jpg"

clip_model = CLIPInstance()

Loading model and processor form local storage


In [2]:
model = YOLO("detection/best.pt")

result = model(image_path)

obb = result[0].obb

xywhr = obb.xywhr
cls = obb.cls

data = []

# # Verify if the export folder exists
# if not os.path.exists(f"{export_path}/{filename}"):

# Create folder to store crops
os.makedirs(f"{export_path}/{filename}", exist_ok=True)

# Save crops and create metadata
for i in range(len(xywhr)):
    data.append(DataFormat(f"{filename}_{i}", xywhr[i], cls[i]))
    image = extract_image(image_path, xywhr[i])

    image.save(f"{export_path}/{filename}/{filename}_{i}.jpg")

for d in data:
    if d.type == 3:
        d.text = extract_text(f"{export_path}/{filename}/{d.filename}.jpg")
    

# Create json file to store crops metadata
with open(f"{export_path}/{filename}/{filename}.json", "w") as f:
    json.dump({d.filename: d.to_dict() for d in data}, f, indent=4)

/home/danieltm/anaconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647329220/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


image 1/1 /home/danieltm/Documents/MATCOM/4A 1S/ML/ML-Facticia/src/../dataset/facticia_3_2/images/4c76a3e4-ohcbh_cf_erl8_204.jpg: 1024x704 190.1ms
Speed: 7.7ms preprocess, 190.1ms inference, 2.1ms postprocess per image at shape (1, 3, 1024, 704)


# Similarity 

In [ ]:
images = os.listdir(export_path)

for filename in images:
    json_file = f"{export_path}/{filename}/{filename}.json"
    with open(json_file, "r") as f:
        data = json.load(f)
        
    crops = []
    for value in data.values():
        crop_name = value["filename"]
        crop_path = f"{export_path}/{filename}/{crop_name}.jpg"
        crop_type = value["type"]
        crop_text = value["text"]
        crop_xywhr = [float(value["x"]), float(value["y"]), float(value["w"]), float(value["h"]), float(value["r"])]    
        crop = DataFormat(crop_name, crop_xywhr, crop_type, crop_text)
        
        crops.append(crop)        
        
    images = [f"{export_path}/{filename}/{crop.filename}.jpg" for crop in crops if crop.type == 2]
    texts = [crop.text for crop in crops if crop.type == 3]
    
    clip_model.get_relation(images[0], texts)
    

[[   0.074274     0.36429     0.55656   0.0047802  0.00010222]]
['LA REMODELACION\nDE LA HABANA\n', 'Vista Genreal de la Alameda de Paula,\ndespués de su restauración.\n', 'Vista general de la Plaza de Agua Dulce.\n', 'Vista del Paso Superior construido por\n\nla Dirección Ceneral de Ingemieros del\n\nMinisterio de Obras Públicas en la barriada\nde Luyano.\n', '- a a\nSN A a\n']
